# Noise Model and Noisy Simulations

*What you will learn:*

- what are the noises in neutral atom QPUs;
- how are they described in a `NoiseModel`;
- how to make noisy simulations of a `Sequence` on an emulator `Backend`.

## `NoiseModel`: Describing the noises in cold-atom QPUS

Cold atom QPUs are subject to various noises, that are going to make the outcome of the execution of your pulser `Sequence` on QPUs different from their theoretical result. The `NoiseModel` class describes the noises to take into account in a neutral atom QPU. It takes as input parameters that characterize each of these noises.

If you have a `NoiseModel`, you can know the noises that it implements checking its property `noise_types`.

## How to include noises in your simulations

Taking into account noises in your simulations is important when designing your pulser `Sequence`, for the outcome of the experiment on the QPU to be close to what you expected. Here is a step-by-step guide on how to run noisy simulations using pulser. It assumes you have already read the [step-by-step guide](./tutorials/backends.nblink) on executing a pulser `Sequence` on a pulser `Backend`.

### 1. Choosing the type of backend

Simulations are performed by using an **Emulator**, that can be **local or remote**.

**Preparation for noisy simulations:**

Noises are associated with QPUs. You need a **remote connection** to connect with a QPU. The list of QPUs associated with a remote connection can be obtained through `connection.fetch_available_devices()`. Using this method, you can get the `Device` associated with the QPU. The [Device](./hardware.ipynb) stores a `NoiseModel` in its `noise_model` attribute. You can take into account the noises during the design of your sequence by checking the values of the noise parameters of this `NoiseModel`

### 2. Creating the pulse Sequence

The next step is to create the sequence that we want to execute. If you want to take into account all the limitations of the QPU, it is best to use the `Device` associated with the QPU when writing your `Sequence`.

### 3. Starting the Backend

An Emulator backend takes as input:
- the `Sequence` to simulate, as all the backends.
- a `RemoteConnection` if the emulation backend is a remote backend.
- an `EmulatorConfig`, that sets the parameters of the emulation. This field is optional, emulator backends have a default config.

The `EmulatorConfig` contains two parameters that configure the noise in the simulation:
- prefer_device_noise_model: Whether or not to use the noise model of the device of the Sequence. By default, it is False. If you defined your Sequence using the Device of a QPU, set this parameter to True to use automatically the noise model of this QPU.
- noise_model: A specific `NoiseModel` defining the noise to include in the simulation, if prefer_device_noise_model is False. By default, this `NoiseModel` does not include any noise. If you want to use a different NoiseModel for your simulation than the `NoiseModel` of your Sequence's Device, you can provide it here. Possible usecases: 
    - You have used a [VirtualDevice](./tutorials/virtual_devices.nblink) for your Sequence, and now want to include noise.
    - You would like to see how the noise of another QPU would impact your Sequence.
    - You would like to see the influence of a certain noise on the execution of your Sequence. NoiseModels are python `dataclasses`, you can modify them with `dataclasses.replace`. For instance, to delete a certain noise parameter "attr", you can do `dataclasses.replace(noise_model, attr=None)`.

### 4. Execution

The execution is always done via the `run` method, using `job_params` to define the number of `runs` performed. Some noises are stochastic: for each run defined by `job_params`, multiple simulations are performed, and the stored result is the average outcome of these simulations. The number of simulations performed per Job's run is determined by `NoiseModel.runs`. If you want to sample multiple times each of these sub-simulations, you can set `NoiseModel.samples_per_run` to more than 1. 



### 5. Retrieving the Results

The returned results are sequence of `Results` objects, containing one `Results` object per `job_params`. Each of these `Results` objects contain `job_params.runs` outcomes.